In [1]:
import numpy as np

import pandas as pd

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import LSTM, Dense, Dropout

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split


In [2]:
data = pd.read_csv('/kaggle/input/rain-forest/MERGE_DATA_HANDING_NULL.csv')  # Thay thế đường dẫn cho phù hợp
data.head()


,lat,lon,time,AWS,ERA5_CAPE,ERA5_CIN,ERA5_EWSS,ERA5_IE,ERA5_ISOR,ERA5_KX,...,ERA5_SLHF,ERA5_SLOR,ERA5_SSHF,ERA5_TCLW,ERA5_TCW,ERA5_TCWV,ERA5_U250,ERA5_U850,ERA5_V250,ERA5_V850
0,0,104,4/1/2019 0:00,0.0,191.375,8.750007,-421.11328,-0.000011,0.676296,37.246895,...,-79335,0.023543,-27706,0.789246,44.727844,43.830704,27.914581,-2.460190,5.097671,7.306610
1,0,106,4/1/2019 0:00,0.0,191.375,8.750007,-421.11328,-0.000011,0.676296,37.246895,...,-79335,0.023543,-27706,0.789246,44.727844,43.830704,27.914581,-2.460190,5.097671,7.306610
2,0,114,4/1/2019 0:00,0.0,446.125,38.875008,-641.11330,-0.000014,0.588248,37.360176,...,-127783,0.019673,-11962,0.723206,49.239563,48.219376,25.793488,-3.606674,4.523453,7.328094
3,0,116,4/1/2019 0:00,0.0,624.125,79.875010,-468.11328,-0.000026,0.315002,35.903145,...,-207399,0.008039,28742,0.581421,48.919250,48.055313,28.781769,-2.479721,4.257828,7.910126
4,0,118,4/1/2019 0:00,0.0,624.125,79.875010,-468.11328,-0.000026,0.315002,35.903145,...,-207399,0.008039,28742,0.581421,48.919250,48.055313,28.781769,-2.479721,4.257828,7.910126


In [3]:
data['time'] = pd.to_datetime(data['time'], format="%m/%d/%Y %H:%M")

data = data.sort_values(by=['lat', 'lon', 'time']).reset_index(drop=True)

data['rainfall'] = data.groupby(['lat', 'lon'])['AWS'].shift(-1)

data = data.dropna(subset=['rainfall']).reset_index(drop=True)

data.head(5)

,lat,lon,time,AWS,ERA5_CAPE,ERA5_CIN,ERA5_EWSS,ERA5_IE,ERA5_ISOR,ERA5_KX,...,ERA5_SLOR,ERA5_SSHF,ERA5_TCLW,ERA5_TCW,ERA5_TCWV,ERA5_U250,ERA5_U850,ERA5_V250,ERA5_V850,rainfall
0,0,104,2019-04-01 00:00:00,0.0,191.375,8.750007,-421.11328,-0.000011,0.676296,37.246895,...,0.023543,-27706,0.789246,44.727844,43.830704,27.914581,-2.460190,5.097671,7.306610,0.0
1,0,104,2019-04-01 01:00:00,0.0,227.500,19.625004,-378.87500,-0.000016,0.676296,37.902550,...,0.023543,-36935,0.707458,42.949050,42.179367,29.826248,-1.982025,6.813675,7.134613,0.0
2,0,104,2019-04-01 02:00:00,0.0,379.500,15.172320,-283.89062,-0.000036,0.676296,36.568780,...,0.023543,-77658,0.655518,42.154434,41.482574,30.636550,-2.467102,7.072754,7.239365,0.0
3,0,104,2019-04-01 03:00:00,0.0,418.625,24.265640,-333.39453,-0.000039,0.676296,34.334260,...,0.023543,-82772,0.643494,41.510223,40.828720,30.549759,-2.507828,6.906708,7.043274,0.0
4,0,104,2019-04-01 04:00:00,0.0,394.250,117.203140,-307.91797,-0.000039,0.676296,31.757630,...,0.023543,-111037,0.673950,40.850906,40.138490,30.195663,-2.724503,6.585526,6.760071,0.0


In [4]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(n_steps, len(data), 1):  
        X.append(data[i - n_steps:i, :-1])  
        y.append(data[i, -1])              
    return np.array(X), np.array(y)

n_steps = 6

X_all, y_all = [], []

cnt = 0

for (lat, lon), group_data in data.groupby(['lat', 'lon']):
    if cnt >=80:
        break
    features = group_data[['AWS','ERA5_TCW', 'ERA5_U850', 'ERA5_EWSS', 'ERA5_V850', 'ERA5_TCLW', 
                           'ERA5_U250', 'ERA5_R850', 'ERA5_R500', 'ERA5_CAPE', 'ERA5_KX', 
                           'ERA5_V250', 'ERA5_R250', 'rainfall']]
    X_group, y_group = prepare_data(features.values, n_steps)
    
    X_all.append(X_group)
    y_all.append(y_group)
    cnt+=1

X_all = np.concatenate(X_all, axis=0)
y_all = np.concatenate(y_all, axis=0)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[2])).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam

def build_model(n_steps, input_dim):
    model = Sequential()
    model.add(LSTM(64, activation='relu', return_sequences=True, input_shape=(n_steps, input_dim)))
    model.add(Dropout(0.2))
    model.add(LSTM(64, activation='relu', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    optimizer = Adam(learning_rate=0.001)

    model.compile(optimizer=optimizer, loss='mse')
    return model

input_dim = X_train.shape[2]  # Số lượng đặc trưng đầu vào
global_model = build_model(n_steps, input_dim)

history = global_model.fit(X_train_scaled, y_train, epochs=32, batch_size=64, validation_data=(X_test_scaled, y_test), verbose=1)

loss = global_model.evaluate(X_test_scaled, y_test, verbose=0)
y_pred = global_model.predict(X_test_scaled)

print(f'Loss trên tập kiểm tra: {loss}')


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/32
2800/2800 ━━━━━━━━━━━━━━━━━━━━ 38s 12ms/step - loss: 1.1849 - val_loss: 0.9983
Epoch 2/32
2800/2800 ━━━━━━━━━━━━━━━━━━━━ 34s 12ms/step - loss: 1.2561 - val_loss: 0.9580
Epoch 3/32
2800/2800 ━━━━━━━━━━━━━━━━━━━━ 34s 12ms/step - loss: 1.1812 - val_loss: 0.9717
Epoch 4/32
2800/2800 ━━━━━━━━━━━━━━━━━━━━ 34s 12ms/step - loss: 1.0210 - val_loss: 0.9249
Epoch 5/32
2800/2800 ━━━━━━━━━━━━━━━━━━━━ 34s 12ms/step - loss: 1.1532 - val_loss: 0.9103
Epoch 6/32
2800/2800 ━━━━━━━━━━━━━━━━━━━━ 33s 12ms/step - loss: 1.1507 - val_loss: 0.9175
Epoch 7/32
2800/2800 ━━━━━━━━━━━━━━━━━━━━ 33s 12ms/step - loss: 1.1059 - val_loss: 0.8842
Epoch 8/32
2800/2800 ━━━━━━━━━━━━━━━━━━━━ 33s 12ms/step - loss: 1.0972 - val_loss: 0.9001
Epoch 9/32
2800/2800 ━━━━━━━━━━━━━━━━━━━━ 34s 12ms/step - loss: 1.0169 - val_loss: 0.9204
Epoch 10/32
2800/2800 ━━━━━━━━━━━━━━━━━━━━ 36s 13ms/step - loss: 1.0438 - val_loss: 0.9167
Epoch 11/32
2800/2800 ━━━━━━━━━━━━━━━━━━━━ 37s 13ms/step - loss: 1.0880 - val_loss: 0.8688
Epoch 12

In [7]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'MSE trên tập kiểm tra: {mse}')
print(f'MAE trên tập kiểm tra: {mae}')

MSE trên tập kiểm tra: 0.8197823755201356
MAE trên tập kiểm tra: 0.1737057643069165
